# 📦 Phase 1 — Data Preparation
## Olist Pricing Intelligence

---

**Objectif :** Charger, nettoyer, joindre et enrichir les tables du dataset Olist Brazilian E-commerce pour produire un dataset analytique unique, prêt à l'emploi pour les analyses de pricing (Phase 2) et le dashboard Power BI (Phase 3).

**Dataset source :** [Olist Brazilian E-commerce — Kaggle](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

> ⚠️ **Prérequis :** télécharger les CSV depuis Kaggle et les placer dans `data/raw/` avant d'exécuter ce notebook.

**Tables utilisées :**

| Table | Rôle |
|---|---|
| `olist_orders_dataset` | Commandes, statuts, dates |
| `olist_order_items_dataset` | Prix, fret, produit par ligne |
| `olist_products_dataset` | Caractéristiques produit |
| `olist_product_category_name_translation` | Traduction catégories (PT → EN) |
| `olist_order_reviews_dataset` | Avis et scores clients |
| `olist_sellers_dataset` | Localisation vendeurs |
| `olist_customers_dataset` | État géographique du client |
| `olist_order_payments_dataset` | Mode de paiement, versements |

**Tables exclues volontairement :**

| Table | Raison |
|---|---|
| `olist_geolocation_dataset` | Coordonnées GPS par code postal — hors scope pricing |

**Output :** `data/processed/olist_pricing_dataset.csv` + `olist_pricing_dataset.xlsx`

---

## Table des matières

1. [Imports & Configuration](#1-imports)
2. [Chargement des données](#2-chargement)
3. [Contrôle qualité](#3-qualite)
4. [Nettoyage](#4-nettoyage)
5. [Jointures](#5-jointures)
6. [Feature Engineering](#6-features)
7. [Visualisations de validation](#7-viz)
8. [Export](#8-export)

---
## 1. Imports & Configuration

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import os
import warnings
warnings.filterwarnings("ignore")

# ── Style global ──────────────────────────────────────────
plt.rcParams.update({
    "figure.facecolor":  "#F8F9FA",
    "axes.facecolor":    "#FFFFFF",
    "axes.spines.top":   False,
    "axes.spines.right": False,
    "font.family":       "sans-serif",
    "axes.titlesize":    13,
    "axes.titleweight":  "bold",
    "axes.labelsize":    11,
    "xtick.labelsize":   10,
    "ytick.labelsize":   10,
})

# ── Palette Okabe-Ito (daltonisme-friendly, standard scientifique) ──
# Source : Okabe & Ito (2008) — https://jfly.uni-koeln.de/color/
PALETTE = [
    "#E69F00",  # Orange
    "#56B4E9",  # Bleu ciel
    "#009E73",  # Vert
    "#F0E442",  # Jaune
    "#0072B2",  # Bleu foncé
    "#D55E00",  # Vermillon
    "#CC79A7",  # Rose/mauve
    "#000000",  # Noir
]

print("✅ Configuration OK — Palette Okabe-Ito chargée")
print(f"   Couleurs disponibles : {len(PALETTE)}")

# ── Chemins ───────────────────────────────────────────────
DATA_RAW = "../data/raw/"
DATA_OUT = "../data/processed/"
IMG_OUT  = "../outputs/"

os.makedirs(DATA_OUT, exist_ok=True)
os.makedirs(IMG_OUT,  exist_ok=True)

# ── Taux de change ────────────────────────────────────────
# Mettre à jour avec le taux souhaité avant exécution
# Source de référence : https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-brl.en.html
BRL_TO_EUR = 0.16   # 1 R$ = 0.17 € (taux indicatif — à ajuster)

print("✅ Configuration OK")
print(f"   Taux de change appliqué : 1 R$ = {BRL_TO_EUR} €")

✅ Configuration OK — Palette Okabe-Ito chargée
   Couleurs disponibles : 8
✅ Configuration OK
   Taux de change appliqué : 1 R$ = 0.16 €


---
## 2. Chargement des données

In [15]:
def load_csv(filename):
    """Charge un CSV depuis data/raw/. Lève une erreur explicite si le fichier est absent."""
    path = os.path.join(DATA_RAW, filename)
    if not os.path.exists(path):
        raise FileNotFoundError(
            f"Fichier introuvable : {path}\n"
            f"  → Télécharge le dataset sur https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce\n"
            f"  → Place les CSV dans le dossier data/raw/"
        )
    df = pd.read_csv(path)
    print(f"  ✅ {filename:<55} {len(df):>8,} lignes")
    return df

print("Chargement des tables Olist...\n")
orders      = load_csv("olist_orders_dataset.csv")
order_items = load_csv("olist_order_items_dataset.csv")
products    = load_csv("olist_products_dataset.csv")
categories  = load_csv("product_category_name_translation.csv")
reviews     = load_csv("olist_order_reviews_dataset.csv")
sellers     = load_csv("olist_sellers_dataset.csv")
customers   = load_csv("olist_customers_dataset.csv")
payments    = load_csv("olist_order_payments_dataset.csv")

Chargement des tables Olist...

  ✅ olist_orders_dataset.csv                                  99,441 lignes
  ✅ olist_order_items_dataset.csv                            112,650 lignes
  ✅ olist_products_dataset.csv                                32,951 lignes
  ✅ product_category_name_translation.csv                         71 lignes
  ✅ olist_order_reviews_dataset.csv                           99,224 lignes
  ✅ olist_sellers_dataset.csv                                  3,095 lignes
  ✅ olist_customers_dataset.csv                               99,441 lignes
  ✅ olist_order_payments_dataset.csv                         103,886 lignes


---
## 3. Analyse Exploratoire des Données (EDA)

EDA des valeurs manquantes, des types et des plages de valeurs sur les colonnes clés.

In [16]:
def quality_report(df, name):
    """Affiche un rapport de qualité concis pour un DataFrame."""
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    print(f"\n{'─'*55}")
    print(f"  {name} — {len(df):,} lignes × {df.shape[1]} colonnes")
    if len(missing):
        print("  Valeurs manquantes :")
        for col, cnt in missing.items():
            print(f"    • {col}: {cnt:,} ({cnt/len(df)*100:.1f}%)")
    else:
        print("  Valeurs manquantes : ✅ aucune")

for df, name in [
    (orders,      "orders"),
    (order_items, "order_items"),
    (products,    "products"),
    (reviews,     "reviews"),
    (customers,   "customers"),
    (payments,    "payments"),
]:
    quality_report(df, name)


───────────────────────────────────────────────────────
  orders — 99,441 lignes × 8 colonnes
  Valeurs manquantes :
    • order_approved_at: 160 (0.2%)
    • order_delivered_carrier_date: 1,783 (1.8%)
    • order_delivered_customer_date: 2,965 (3.0%)

───────────────────────────────────────────────────────
  order_items — 112,650 lignes × 7 colonnes
  Valeurs manquantes : ✅ aucune

───────────────────────────────────────────────────────
  products — 32,951 lignes × 9 colonnes
  Valeurs manquantes :
    • product_category_name: 610 (1.9%)
    • product_name_lenght: 610 (1.9%)
    • product_description_lenght: 610 (1.9%)
    • product_photos_qty: 610 (1.9%)
    • product_weight_g: 2 (0.0%)
    • product_length_cm: 2 (0.0%)
    • product_height_cm: 2 (0.0%)
    • product_width_cm: 2 (0.0%)

───────────────────────────────────────────────────────
  reviews — 99,224 lignes × 7 colonnes
  Valeurs manquantes :
    • review_comment_title: 87,656 (88.3%)
    • review_comment_message: 58,247 (

In [18]:
# Distribution des statuts de commande
print("Distribution des statuts de commande :")
print(orders["order_status"].value_counts(normalize=True).mul(100).round(1).to_string())

Distribution des statuts de commande :
order_status
delivered      97.0
shipped         1.1
canceled        0.6
unavailable     0.6
invoiced        0.3
processing      0.3
created         0.0
approved        0.0


In [19]:
# Distribution des modes de paiement
print("Distribution des modes de paiement :")
print(payments["payment_type"].value_counts(normalize=True).mul(100).round(1).to_string())

print("\nStatistiques — nombre de versements :")
print(payments["payment_installments"].describe().round(1).to_string())

Distribution des modes de paiement :
payment_type
credit_card    73.9
boleto         19.0
voucher         5.6
debit_card      1.5
not_defined     0.0

Statistiques — nombre de versements :
count    103886.0
mean          2.9
std           2.7
min           0.0
25%           1.0
50%           1.0
75%           4.0
max          24.0


In [20]:
# Plages de prix — détection d'anomalies
print("Statistiques descriptives — prix & fret :")
order_items[["price", "freight_value"]].describe().round(2)

Statistiques descriptives — prix & fret :


,price,freight_value
count,112650.00,112650.00
mean,120.65,19.99
std,183.63,15.81
min,0.85,0.00
25%,39.90,13.08
50%,74.99,16.26
75%,134.90,21.15
max,6735.00,409.68
